<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import re 
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


#import warnings
#warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ar_ma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ar_ma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ar_ma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from sklearn.model_selection import GridSearchCV

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
from sklearn.metrics import f1_score

In [3]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)

# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [4]:
#df_tc = pd.read_csv('/datasets/toxic_comments.csv')

df_tc = pd.read_csv('toxic_comments.csv')

Изучим данные:

In [5]:
df_tc.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [6]:
df_tc['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Классы несбалансированы. Если предсказывать класс "0" всегда, то точность будет 89,8%, поэтому необходимо учитывать класс 1 с бОльшим весом.

In [7]:
df_tc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Очистим текст от знаков препинания и проведем лемматизацию:

In [8]:
def clean_text(text):
    """
    Все заглавные буквы становятся строчными, текст очищается от знаков препинания
    """
    text = text.lower()
    pre_clean_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clean_text = " ".join(pre_clean_text.split())
    return clean_text
    
m = WordNetLemmatizer()

def lemmatize(text):
    """
    Приведение слов к леммам
    """
    word_list = nltk.word_tokenize(text)
    return ' '.join([m.lemmatize(w) for w in word_list])

In [9]:
%%time

df_tc['lemm_text'] = df_tc['text'].apply(clean_text)

Wall time: 4.33 s


In [10]:
%%time

df_tc['lemm_text'] = df_tc['lemm_text'].apply(lemmatize)

Wall time: 1min 12s


In [11]:
corpus = df_tc['lemm_text'].values.astype('U')

Нужно разбить выборки перед тем, как создавать мешок слов:

In [12]:
features = corpus
target = df_tc['toxic']

In [13]:
train_features, test_features, train_target, test_target = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.25,
                                                                            stratify=target,
                                                                            random_state=12345)

In [14]:
stopwords = set(stopwords.words('english'))

### Вывод

Был загружен и изучен датасет с токсичными комментариями, но классы не сбалансированы.
Текстовый признак был очищен и приведен к леммам, выборка была разбита на обучающую и тестовую, а также вычислен TF-IDF для корпуса текстов.

## Обучение

### LogisticRegression

Подберем гиперпараметры для модели логистической регрессии:

In [15]:
%%time

pipe_lr = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345))])

grid_params_lr = [{'clf__penalty': ['l1', 'l2', 'none'],
                   'clf__C': np.logspace(-4, 4, 20),
                   'clf__solver': ['liblinear']}] 

LR = GridSearchCV(estimator=pipe_lr,
                  param_grid=grid_params_lr, scoring='f1', n_jobs=-1, cv=4) 
LR.fit(train_features, train_target)

print("F1 логистической регрессии составляет ", LR.best_score_)
print('Наилучшие подобранные параметры для логистической регрессии')
print(LR.best_params_)

C:\Users\ar_ma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
80 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ar_ma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ar_ma\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\ar_ma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty,

F1 логистической регрессии составляет  0.7785725224765353
Наилучшие подобранные параметры для логистической регрессии
{'clf__C': 4.281332398719396, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
Wall time: 7min 29s


### DecisionTreeClassifier

In [22]:
%%time

pipe_dt = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', DecisionTreeClassifier(random_state=12345))])

param_range = [15, 25]
grid_params_dt = [{'clf__max_depth': param_range}]

DT = GridSearchCV(estimator=pipe_dt,
                  param_grid=grid_params_dt, n_jobs=-1, scoring='f1', cv=4) 
DT.fit(train_features, train_target)

print("F1 для DT составляет ", DT.best_score_)
print('Наилучшие подобранные параметры для DT')
print(DT.best_params_)

F1 для RF составляет  0.6657513122661067
Наилучшие подобранные параметры для DT
{'clf__max_depth': 25}
Wall time: 48.7 s


### RandomForestClassifier

In [31]:
%%time

pipe_rf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', RandomForestClassifier(random_state=12345, ))])

param_range = [15, 25]
grid_params_rf = [{'clf__max_depth': param_range,
                   'clf__n_estimators':[100, 200, 300],
                   'clf__class_weight': ["balanced"]}]

RF = GridSearchCV(estimator=pipe_rf,
                  param_grid=grid_params_rf, n_jobs=-1, scoring='f1', cv=4) 
RF.fit(train_features, train_target)

print("F1 для RF составляет ", RF.best_score_)
print('Наилучшие подобранные параметры для RF')
print(RF.best_params_)

F1 для RF составляет  0.43101372091972134
Наилучшие подобранные параметры для RF
{'clf__class_weight': 'balanced', 'clf__max_depth': 25, 'clf__n_estimators': 300}
Wall time: 2min 51s


### Вывод

Были подобраны гиперпараметры для 3-х моделей, значение F-1 на кросс-валидации следующее:
* LogisticRegression 0.7786
* DecisionTreeClassifier 0.6658
* RandomForestClassifier 0.431

Лучше всего по значению F-1 показала себя логистическая регрессия.

## Выводы

### LogisticRegression

In [35]:
%%time

pipe_lr2 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345, C=4.281332398719396, penalty='l1', solver='liblinear', n_jobs=-1))])
pipe_lr2.fit(train_features, train_target)
predicted_lr = pipe_lr2.predict(test_features)
print(f1_score(predicted_lr, test_target))

C:\Users\ar_ma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  warnings.warn(


0.7790729194903768
Wall time: 9.56 s


### DecisionTreeClassifier

In [37]:
%%time

pipe_dt2 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', DecisionTreeClassifier(max_depth=25, random_state=12345))])
pipe_dt2.fit(train_features, train_target)
predicted_dt = pipe_dt2.predict(test_features)
print(f1_score(predicted_dt, test_target))

0.6638850254983774
Wall time: 19.3 s


### RandomForestClassifier

In [38]:
%%time

pipe_rf2 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf',  RandomForestClassifier(max_depth=25, random_state=12345, class_weight='balanced', n_estimators=300))])
pipe_rf2.fit(train_features, train_target)
predicted_rf = pipe_rf2.predict(test_features)
print(f1_score(predicted_rf, test_target))

0.42105263157894746
Wall time: 42.3 s


### Вывод

Логистическая регрессия показала себя лучше, как по метрике F-1 (0.7791), так и по скорости обучения и предсказания.

## Вывод

Признаки, подготовленные с помощью TfidfVectorizer и использованные на модели логистической регрессии показали лучший требуемый результат F-1: 0,7791, в то время как дерево решений и случайный лес не справились с поставленной задачей (случайный лес).